In [1]:
using Gridap
using Gridap.Algebra;
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData

using GridapGmsh

using LinearAlgebra, ForwardDiff
using FillArrays, SparseArrays

# Geometry

In [2]:
# Enclosure dimensions (enclosure is not centered, the core is)
hencl1 = 65.5e-2;   # Height above the x-axis
hencl2 = -53.5e-2;  # Height below the x-axis
wencl  = 104e-2;    # Width

# Core dimensions
wcore = 84e-2;
hcore = 100e-2;

# Core gap dimensions (left and right are identical)
wgap = 17e-2;
hgap = 76e-2;
mgap = 17e-2;

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Mesh densities
lc1 = 2e-2;      # Enclosure & core outer
lc2 = 1e-2;      # Core inner
lc3 = 1e-2;      # HV windings
lc4 = 1e-2;      # LV windings

# Gridap Setup

In [3]:
# Define norm of a complex vector for Gridap to use on the CellField which results from Bh = ∇(uh)
norm_complex(a::VectorValue{2, ComplexF64}) = sqrt(norm(a[1])^2 + norm(a[2])^2)
norm_complex(a::CellField) = Operation(norm_complex)(a)

norm_complex (generic function with 2 methods)


(process:15852): GLib-GIO-WARNING **: 12:02:33.017: Unexpectedly, UWP app `SiemensPLMSoftware.JT2GoDesktop_14.1.1.0_x64__qtbmxjdagz8xc' (AUMId `SiemensPLMSoftware.JT2GoDesktop_qtbmxjdagz8xc!JT2GoDesktop') supports 18 extensions but has no verbs


# Linear BH Curve

In [ ]:
# Load mesh
model = GmshDiscreteModel("geo/transformer_stedin.msh")
Ω = Triangulation(model)

# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)
dΩ = Measure(Ω, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
V = TestFESpace(model, reffe, conformity = :H1, dirichlet_tags = ["Enclosure"], vector_type = Vector{ComplexF64})
U = TrialFESpace(V, [0])

In [ ]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

mu0 = 4e-7 * pi;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

In [ ]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Ω)

const tag_core = get_tag_from_name(labels, "Core");

const tag_lv1l = 10; const tag_lv1r = 11;
const tag_lv2l = 12; const tag_lv2r = 13;
const tag_lv3l = 14; const tag_lv3r = 15;

# Source current density
function fsource(tag)
    if tag == tag_lv1l        # LV winding phase 1 left
        return Js * exp(1im * 2pi/3);
    elseif tag == tag_lv1r    # LV winding phase 1 right
        return -Js * exp(1im * 2pi/3);
    elseif tag == tag_lv2l    # LV winding phase 2 left
        return Js;
    elseif tag == tag_lv2r    # LV winding phase 2 right
        return -Js;
    elseif tag == tag_lv3l    # LV winding phase 3 left
        return Js * exp(-1im * 2pi/3);
    elseif tag == tag_lv3r    # LV winding phase 3 right
        return -Js * exp(-1im * 2pi/3);
    else
        return 0.0 + 0.0im;
    end
end

# Permeability model
function fmur(tag)
    if tag == tag_core
        return 1000.0;
    else
        return 1.0;
    end
end

function fnu(tag)
    return 1 / (mu0 * fmur(tag));
end

# Conductivity
function fsigma(tag)
    if tag == tag_core
        return 0.1;
    else
        return 0.0;
    end
end

In [ ]:
# Define the weak form (bilinear and linear terms)
a(u,v) = ∫( (fnu ∘ τ) ⋅ ∇(u) ⋅ ∇(v) )dΩ + ∫( 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ u ⋅ v )dΩ
b(v)   = ∫( (fsource ∘ τ) ⋅ v )dΩ;
op = AffineFEOperator(a, b, U, V)

# Solve the linear FE system with LU solver
ls = LUSolver()
solver = LinearFESolver(ls)
uh = solve(solver, op);

In [ ]:
# Post-processing for magnetic field and current density
Bh = norm_complex(gradient(uh));

J0    = (fsource ∘ τ); # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [ ]:
writevtk(Ω, "images/transformer_gridap/transformer1", cellfields=["Az"=>abs(uh), "imAz"=>imag(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy)])

![Result: Magnetic Flux Density](images/transformer_gridap/transformer1.png)

# Non-linear BH Curve

In [ ]:
# Load mesh
model = GmshDiscreteModel("geo/transformer_stedin.msh")
#model = GmshDiscreteModel("geo/transformer_stedin_hybrid.msh")    # This works if GridapGmsh is modified to allow multiple element types per dimension
Ω = Triangulation(model)

# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)
dΩ = Measure(Ω, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
V = TestFESpace(model, reffe, conformity = :H1, dirichlet_tags = ["Enclosure"], vector_type = Vector{ComplexF64})
U = TrialFESpace(V, [0])

In [ ]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

mu0 = 4e-7 * pi;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

In [ ]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Ω)

const tag_core = get_tag_from_name(labels, "Core");

const tag_lv1l = 10; const tag_lv1r = 11;
const tag_lv2l = 12; const tag_lv2r = 13;
const tag_lv3l = 14; const tag_lv3r = 15;

# Source current density
function fsource(tag)
    if tag == tag_lv1l        # LV winding phase 1 left
        return Js * exp(1im * 2pi/3);
    elseif tag == tag_lv1r    # LV winding phase 1 right
        return -Js * exp(1im * 2pi/3);
    elseif tag == tag_lv2l    # LV winding phase 2 left
        return Js;
    elseif tag == tag_lv2r    # LV winding phase 2 right
        return -Js;
    elseif tag == tag_lv3l    # LV winding phase 3 left
        return Js * exp(-1im * 2pi/3);
    elseif tag == tag_lv3r    # LV winding phase 3 right
        return -Js * exp(-1im * 2pi/3);
    else
        return 0.0 + 0.0im;
    end
end

# Permeability model
bh_a = 2.12e-4; 
bh_b = 7.358;
bh_c = 1.18e7;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));
fdmur_core(B) = ForwardDiff.derivative(fmur_core, B);

function fmur(tag, ∇u)
    if tag == tag_core
        normB = norm_complex(∇u);
        return fmur_core(normB);
    else
        return 1.0;
    end
end

function fdmur(tag, ∇du, ∇u)
    if tag == tag_core
        normB = norm_complex(∇u);
        return fdmur_core(normB) ⋅ (∇du ⊙ ∇u);
    else
        return 0.0 + 0.0im;
    end
end

function fnu(tag, ∇u)
    return 1 / (mu0 * fmur(tag, ∇u));
end

function fdnu(tag, ∇du, ∇u)
    return -1 / (mu0 * fmur(tag, ∇u)^2) * fdmur(tag, ∇du, ∇u);
end

# Conductivity
function fsigma(tag)
    if tag == tag_core
        return 1.0;
    else
        return 0.0;
    end
end

In [ ]:
# Define the weak form (bilinear and linear terms)
res(u,v) = ∫( (fnu ∘ (τ, ∇(u))) ⋅ ∇(u) ⋅ ∇(v) )dΩ + ∫( 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ u ⋅ v )dΩ - ∫( (fsource ∘ τ) ⋅ v )dΩ;
jac(u, du, v) = ∫( (fdnu ∘ (τ, ∇(du), ∇(u))) ⋅ ∇(u) ⋅ ∇(v) )dΩ + ∫( (fnu ∘ (τ, ∇(u))) ⋅ ∇(du) ⋅ ∇(v) )dΩ + ∫( 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ du ⋅ v )dΩ;;
op = FEOperator(res, jac, U, V)

ls = BackslashSolver()
nls = NLSolver(ls, method = :trust_region, show_trace = true)
solver = FESolver(nls)

x   = zeros(Complex{Float64}, num_free_dofs(U))
uh0 = FEFunction(U, x)
uh, = solve!(uh0, solver, op)

In [ ]:
# Post-processing for magnetic field and current density
Bh = norm_complex(∇(uh)); # Magnetic flux density
mu_r = fmur ∘ (τ, ∇(uh)); # Relative permeability

J0    = fsource ∘ τ; # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [ ]:
writevtk(Ω, "images/transformer_gridap/transformer2", cellfields=["Az"=>abs(uh), "imAz"=>imag(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy), "mur" => mu_r])

![Result: Magnetic Flux Density](images/transformer_gridap/transformer2.png)

![Result: Relative Permeability](images/transformer_gridap/transformer2_mu.png)

# Using Hybrid Meshes in Gridap
At the moment, ``GridapGmsh`` does not allow meshes with more than one element type per dimension (such as hybrid triangle-quad meshes). However, all of the functionality to allow (simple) hybrid meshes is present. I will try to submit a pull request or open an issue to extend this functionality. We only require changes to internal function ``_setup_reffes``.

In [ ]:
function _setup_reffes(gmsh,d,orient_if_simplex)
    elemTypes, elemTags, nodeTags = gmsh.model.mesh.getElements(d)

    ncells, nmin, nmax = _check_cell_tags(elemTags)
    cell_to_type = fill(Int8(0), ncells)
    
    noffset = nmin - 1;
    for (i, etype) in enumerate(elemTypes)
        cell_to_type[elemTags[i] .- noffset] .= i   # Correctly fill the cell_to_type array, instead of filling '1'
    
        # Check that the elements are first order; higher-order elements are not yet supported
        name, dim, order::Int, numv, parv = gmsh.model.mesh.getElementProperties(etype)
        if order == 0 && etype == POINT
            order = 1
        end
        
        if order != 1
            gmsh.finalize()
            error("For the moment only for first-order elements")
        end
    end
    
    # Retrieve RefFEs for the element types
    reffes = [_reffe_from_etype(etype) for etype in elemTypes];

    # Check mesh orientation
    boo = [(is_simplex(get_polytope(reffe)) && orient_if_simplex) for reffe in reffes];
    orientation = any(boo) ? Oriented() : NonOriented()

    (cell_to_type, reffes, orientation)
end

![Result: Magnetic Flux Density](images/transformer_gridap/transformer4.png)

# Linear BH Curve with Voltage-Driven Coils

In [ ]:
# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)

# Load mesh
model = GmshDiscreteModel("geo/transformer_stedin_hybrid.msh")
Th = Triangulation(model)
Qh = CellQuadrature(Th, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
Vh = TestFESpace(model, reffe; conformity = :H1, dirichlet_tags = ["Enclosure"], vector_type = Vector{ComplexF64})
Uh = TrialFESpace(Vh, [0])

In [ ]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

mu0 = 4e-7 * pi;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

In [ ]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Th)

const tag_core = get_tag_from_name(labels, "Core");
const tag_lv1l = get_tag_from_name(labels, "LV1l");
const tag_lv1r = get_tag_from_name(labels, "LV1r");
const tag_lv2l = get_tag_from_name(labels, "LV2l");
const tag_lv2r = get_tag_from_name(labels, "LV2r");
const tag_lv3l = get_tag_from_name(labels, "LV3l");
const tag_lv3r = get_tag_from_name(labels, "LV3r");

# Source current density
function fsource(tag)
    return 0.0 + 0.0im;
end

# Permeability model
function fmur(tag)
    if tag == tag_core
        return 1000.0;
    else
        return 1.0;
    end
end

function fnu(tag)
    return 1 / (mu0 * fmur(tag));
end

# Conductivity
function fsigma(tag)
    if tag == tag_core
        return 0.1;
    else
        return 0.0;
    end
end

In [ ]:
dv = get_fe_basis(Vh);
du = get_trial_fe_basis(Uh);

# Calculate contributions of the cells to the linear system
cellvals_A = ∫( (fnu ∘ τ) ⋅ ∇(du) ⋅ ∇(dv) + 1im * omega * (fsigma ∘ τ) ⋅ du ⋅ dv )*Qh;
cellvals_f = ∫( (fsource ∘ τ) ⋅ dv )*Qh;

# Assemble the linear system
assem = SparseMatrixAssembler(Uh, Vh)
σk = get_cell_dof_ids(Uh);

rs = ([cellvals_f], [σk]);
b = allocate_vector(assem, rs);
assemble_vector!(b, assem, rs);

rs = ([cellvals_A], [σk], [σk]);
A = allocate_matrix(assem, rs);
assemble_matrix!(A, assem, rs);

In [ ]:
# Circuit parameters
CFF  = 0.3;
Vsec = 420 * sqrt(2 / 3);
Rsec = 1.2999e-3 / CFF;
Asec = Awlv;
lsec = 0.4;

# Extend the linear system with an additional row and column per circuit equation
N  = num_free_dofs(Uh);
nc = 3;

C  = Matrix(UniformScaling(1), nc, nc);
f2 = [ Vsec * exp(1im * 2pi/3) / Rsec;
       Vsec / Rsec;
       Vsec * exp(-1im * 2pi/3) / Rsec
];

# Calculate the cell contributions to the extra vectors
Th_lv1r = Triangulation(model, tags = "LV1r"); Qh_lv1r = CellQuadrature(Th_lv1r, 2*order)
Th_lv1l = Triangulation(model, tags = "LV1l"); Qh_lv1l = CellQuadrature(Th_lv1l, 2*order)
Th_lv2r = Triangulation(model, tags = "LV2r"); Qh_lv2r = CellQuadrature(Th_lv2r, 2*order)
Th_lv2l = Triangulation(model, tags = "LV2l"); Qh_lv2l = CellQuadrature(Th_lv2l, 2*order)
Th_lv3r = Triangulation(model, tags = "LV3r"); Qh_lv3r = CellQuadrature(Th_lv3r, 2*order)
Th_lv3l = Triangulation(model, tags = "LV3l"); Qh_lv3l = CellQuadrature(Th_lv3l, 2*order)

cellvals_lv1lT = ∫( Ns / Asec ⋅ dv )*Qh_lv1l;
cellvals_lv1rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv1r;
cellvals_lv2lT = ∫( Ns / Asec ⋅ dv )*Qh_lv2l;
cellvals_lv2rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv2r;
cellvals_lv3lT = ∫( Ns / Asec ⋅ dv )*Qh_lv3l;
cellvals_lv3rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv3r;

cellvals_lv1l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv1l;
cellvals_lv1r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv1r;
cellvals_lv2l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv2l;
cellvals_lv2r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv2r;
cellvals_lv3l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv3l;
cellvals_lv3r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv3r;

# Assemble the circuit coupling equations into extra vectors
σ_lv1l = get_cell_dof_ids(Uh, Th_lv1l);
σ_lv1r = get_cell_dof_ids(Uh, Th_lv1r);
σ_lv2l = get_cell_dof_ids(Uh, Th_lv2l);
σ_lv2r = get_cell_dof_ids(Uh, Th_lv2r);
σ_lv3l = get_cell_dof_ids(Uh, Th_lv3l);
σ_lv3r = get_cell_dof_ids(Uh, Th_lv3r);

rs = ([cellvals_lv1lT; cellvals_lv1rT], [σ_lv1l; σ_lv1r]);
b1T = allocate_vector(assem, rs);
assemble_vector!(b1T, assem, rs);

rs = ([cellvals_lv2lT; cellvals_lv2rT], [σ_lv2l; σ_lv2r]);
b2T = allocate_vector(assem, rs);
assemble_vector!(b2T, assem, rs);

rs = ([cellvals_lv3lT; cellvals_lv3rT], [σ_lv3l; σ_lv3r]);
b3T = allocate_vector(assem, rs);
assemble_vector!(b3T, assem, rs);

rs = ([cellvals_lv1l; cellvals_lv1r], [σ_lv1l; σ_lv1r]);
b1 = allocate_vector(assem, rs);
assemble_vector!(b1, assem, rs);

rs = ([cellvals_lv2l; cellvals_lv2r], [σ_lv2l; σ_lv2r]);
b2 = allocate_vector(assem, rs);
assemble_vector!(b2, assem, rs);

rs = ([cellvals_lv3l; cellvals_lv3r], [σ_lv3l; σ_lv3r]);
b3 = allocate_vector(assem, rs);
assemble_vector!(b3, assem, rs);

In [ ]:
# Create the modified system \tilde{A} = [A BT; B C] and \tilde{b} = [b1; b2]
At = [A [b1T b2T b3T]; transpose([b1 b2 b3]) C];
bt = [b; f2];

# Solve the modified linear system and package the solution in a FEFunction
u = At \ bt;
uh = FEFunction(Uh, u);

In [ ]:
Isec1 = u[end-2];
Isec2 = u[end-1];
Isec3 = u[end-0];

print("I_{sec1}: ", norm(Isec1), " A, ", angle(Isec1) / pi * 180, " deg\n")
print("I_{sec2}: ", norm(Isec2), " A, ", angle(Isec2) / pi * 180, " deg\n")
print("I_{sec3}: ", norm(Isec3), " A, ", angle(Isec3) / pi * 180, " deg\n")

In [ ]:
# Post-processing for magnetic field and current density
Bh = norm_complex(gradient(uh));

function fsource(tag)
    if tag == tag_lv1l        # LV winding phase 1 left
        return Isec1 / Awlv * exp(1im * 2pi/3);
    elseif tag == tag_lv1r    # LV winding phase 1 right
        return -Isec1 / Awlv * exp(1im * 2pi/3);
    elseif tag == tag_lv2l    # LV winding phase 2 left
        return Isec2 / Awlv;
    elseif tag == tag_lv2r    # LV winding phase 2 right
        return -Isec2 / Awlv;
    elseif tag == tag_lv3l    # LV winding phase 3 left
        return Isec3 / Awlv * exp(-1im * 2pi/3);
    elseif tag == tag_lv3r    # LV winding phase 3 right
        return -Isec3 / Awlv * exp(-1im * 2pi/3);
    else
        return 0.0 + 0.0im;
    end
end

J0    = fsource ∘ τ; # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [ ]:
writevtk(Th, "images/transformer_gridap/transformer5", cellfields=["Az"=>abs(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy)])

![Flux Density](images/transformer_gridap/transformer5.png)

# Non-Linear BH Curve with Voltage-Driven Coils

In [4]:
# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)

# Load mesh
model = GmshDiscreteModel("geo/transformer_stedin_hybrid.msh")
Th = Triangulation(model)
Qh = CellQuadrature(Th, 2*order)

# Construct Lagrangian test space with dirichlet condition on the right boundary node
Vh = TestFESpace(model, reffe; conformity = :H1, dirichlet_tags = ["Enclosure"], vector_type = Vector{ComplexF64})
Uh = TrialFESpace(Vh, [0])

Info    : Reading 'geo/transformer_stedin_hybrid.msh'...
Info    : 194 entities
Info    : 10576 nodes
Info    : 17217 elements
Info    : Done reading 'geo/transformer_stedin_hybrid.msh'


TrialFESpace()

In [5]:
Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

mu0 = 4e-7 * pi;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

In [6]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

# Construct a cell field consisting of the physical domain tags
τ = CellField(tags, Th)

const tag_core = get_tag_from_name(labels, "Core");
const tag_lv1l = get_tag_from_name(labels, "LV1l");
const tag_lv1r = get_tag_from_name(labels, "LV1r");
const tag_lv2l = get_tag_from_name(labels, "LV2l");
const tag_lv2r = get_tag_from_name(labels, "LV2r");
const tag_lv3l = get_tag_from_name(labels, "LV3l");
const tag_lv3r = get_tag_from_name(labels, "LV3r");

# Source current density
function fsource(tag)
    return 0.0 + 0.0im;
end

# Permeability model
bh_a = 1 / 47e3;
bh_b = 3.6;
bh_c = 2.1e8;
mu0  = 4e-7 * pi;
fmur_core(B) = 1 / (bh_a + (1 - bh_a) * B^(2*bh_b) / (B^(2*bh_b) + bh_c));

function fmur(tag, ∇u)
    if tag == tag_core
        normB = norm_complex(∇u);
        return fmur_core(normB);
    else
        return 1.0;
    end
end

function fnu(tag, ∇u)
    return 1 / (mu0 * fmur(tag, ∇u));
end

# Conductivity
function fsigma(tag)
    if tag == tag_core
        return 0.1;
    else
        return 0.0;
    end
end

fsigma (generic function with 1 method)

In [7]:
dv = get_fe_basis(Vh);
du = get_trial_fe_basis(Uh);

# Assemble the linear system
assem = SparseMatrixAssembler(Uh, Vh)
σk = get_cell_dof_ids(Uh);

In [8]:
# Circuit parameters
CFF  = 0.3;
Vsec = 420 * sqrt(2 / 3);
Rsec = 1.2999e-3 / CFF;
Asec = Awlv;
lsec = 0.4;

# Extend the linear system with an additional row and column per circuit equation
N  = num_free_dofs(Uh);
nc = 3;

C  = Matrix(UniformScaling(1), nc, nc);
f2 = [ Vsec * exp(1im * 2pi/3) / Rsec;
       Vsec / Rsec;
       Vsec * exp(-1im * 2pi/3) / Rsec
];

# Calculate the cell contributions to the extra vectors
Th_lv1r = Triangulation(model, tags = "LV1r"); Qh_lv1r = CellQuadrature(Th_lv1r, 2*order)
Th_lv1l = Triangulation(model, tags = "LV1l"); Qh_lv1l = CellQuadrature(Th_lv1l, 2*order)
Th_lv2r = Triangulation(model, tags = "LV2r"); Qh_lv2r = CellQuadrature(Th_lv2r, 2*order)
Th_lv2l = Triangulation(model, tags = "LV2l"); Qh_lv2l = CellQuadrature(Th_lv2l, 2*order)
Th_lv3r = Triangulation(model, tags = "LV3r"); Qh_lv3r = CellQuadrature(Th_lv3r, 2*order)
Th_lv3l = Triangulation(model, tags = "LV3l"); Qh_lv3l = CellQuadrature(Th_lv3l, 2*order)

cellvals_lv1lT = ∫( Ns / Asec ⋅ dv )*Qh_lv1l;
cellvals_lv1rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv1r;
cellvals_lv2lT = ∫( Ns / Asec ⋅ dv )*Qh_lv2l;
cellvals_lv2rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv2r;
cellvals_lv3lT = ∫( Ns / Asec ⋅ dv )*Qh_lv3l;
cellvals_lv3rT = ∫( -Ns / Asec ⋅ dv )*Qh_lv3r;

cellvals_lv1l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv1l;
cellvals_lv1r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv1r;
cellvals_lv2l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv2l;
cellvals_lv2r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv2r;
cellvals_lv3l = ∫( -1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv3l;
cellvals_lv3r = ∫( 1im * omega * Ns * lsec / (Asec * Rsec) ⋅ dv )*Qh_lv3r;

# Assemble the circuit coupling equations into extra vectors
σ_lv1l = get_cell_dof_ids(Uh, Th_lv1l);
σ_lv1r = get_cell_dof_ids(Uh, Th_lv1r);
σ_lv2l = get_cell_dof_ids(Uh, Th_lv2l);
σ_lv2r = get_cell_dof_ids(Uh, Th_lv2r);
σ_lv3l = get_cell_dof_ids(Uh, Th_lv3l);
σ_lv3r = get_cell_dof_ids(Uh, Th_lv3r);

rs = ([cellvals_lv1lT; cellvals_lv1rT], [σ_lv1l; σ_lv1r]);
b1T = allocate_vector(assem, rs);
assemble_vector!(b1T, assem, rs);

rs = ([cellvals_lv2lT; cellvals_lv2rT], [σ_lv2l; σ_lv2r]);
b2T = allocate_vector(assem, rs);
assemble_vector!(b2T, assem, rs);

rs = ([cellvals_lv3lT; cellvals_lv3rT], [σ_lv3l; σ_lv3r]);
b3T = allocate_vector(assem, rs);
assemble_vector!(b3T, assem, rs);

rs = ([cellvals_lv1l; cellvals_lv1r], [σ_lv1l; σ_lv1r]);
b1 = allocate_vector(assem, rs);
assemble_vector!(b1, assem, rs);

rs = ([cellvals_lv2l; cellvals_lv2r], [σ_lv2l; σ_lv2r]);
b2 = allocate_vector(assem, rs);
assemble_vector!(b2, assem, rs);

rs = ([cellvals_lv3l; cellvals_lv3r], [σ_lv3l; σ_lv3r]);
b3 = allocate_vector(assem, rs);
assemble_vector!(b3, assem, rs);

In [9]:
u0 = FEFunction(Uh, zeros(Complex{Float64}, num_free_dofs(Uh)))

SingleFieldFEFunction():
 num_cells: 16993
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 16803535976189124150

In [10]:
# Calculate contributions of the cells to the linear system
cellvals_A = ∫( (fnu ∘ (τ, ∇(u0))) ⋅ ∇(du) ⋅ ∇(dv) + 1im * omega * (fsigma ∘ τ) ⋅ du ⋅ dv )*Qh;
cellvals_f = ∫( (fsource ∘ τ) ⋅ dv )*Qh;

rs = ([cellvals_f], [σk]);
b = allocate_vector(assem, rs);
assemble_vector!(b, assem, rs);

rs = ([cellvals_A], [σk], [σk]);
A = allocate_matrix(assem, rs);
assemble_matrix!(A, assem, rs);

# Create the modified system \tilde{A} = [A BT; B C] and \tilde{b} = [b1; b2]
At = [A [b1T b2T b3T]; transpose([b1 b2 b3]) C];
bt = [b; f2];

# Solve the modified linear system and package the solution in a FEFunction
u = At \ bt;

In [ ]:
deltau = 1e6;
Niter = 1;
alpha = 0.9

tol = 1e-6;
Nmax = 200;

uhist = zeros(Complex{Float64}, N + nc);

while (deltau > tol) && (Niter < Nmax)
    uprev = u;
    uhist = uhist * alpha + u * (1 - alpha);    # Provide some damping to prevent oscillation between two solutions

    uh_n = FEFunction(Uh, uhist)
    
    # Calculate contributions of the cells to the linear system
    cellvals_A = ∫( (fnu ∘ (τ, ∇(uh_n))) ⋅ ∇(du) ⋅ ∇(dv) + 1im * omega * (fsigma ∘ τ) ⋅ du ⋅ dv )*Qh;
    cellvals_f = ∫( (fsource ∘ τ) ⋅ dv )*Qh;
    
    rs = ([cellvals_f], [σk]);
    assemble_vector!(b, assem, rs);

    rs = ([cellvals_A], [σk], [σk]);
    assemble_matrix!(A, assem, rs);

    # Create the modified system \tilde{A} = [A BT; B C] and \tilde{b} = [b1; b2]
    At = [A [b1T b2T b3T]; transpose([b1 b2 b3]) C];
    bt = [b; f2];

    # Solve the modified linear system and package the solution in a FEFunction
    u = At \ bt;

    deltau = norm(u - uprev);
    Niter += 1;
    print("#$Niter: $deltau\n")
end

In [12]:
uh = FEFunction(Uh, uhist);

In [13]:
Isec1 = uhist[end-2];
Isec2 = uhist[end-1];
Isec3 = uhist[end-0];

print("I_{sec1}: ", norm(Isec1), " A, ", angle(Isec1) / pi * 180, " deg\n")
print("I_{sec2}: ", norm(Isec2), " A, ", angle(Isec2) / pi * 180, " deg\n")
print("I_{sec3}: ", norm(Isec3), " A, ", angle(Isec3) / pi * 180, " deg\n")

I_{sec1}: 28.32231746537388 A, -163.21961159263614 deg
I_{sec2}: 15.877738267008876 A, 90.08342343130786 deg
I_{sec3}: 28.319748478502806 A, -15.731307440474106 deg


In [14]:
# Post-processing for magnetic field and current density
Bh = norm_complex(∇(uh));
mu_r = fmur ∘ (τ, ∇(uh)); # Relative permeability

function fsource(tag)
    if tag == tag_lv1l        # LV winding phase 1 left
        return Isec1 / Awlv * exp(1im * 2pi/3);
    elseif tag == tag_lv1r    # LV winding phase 1 right
        return -Isec1 / Awlv * exp(1im * 2pi/3);
    elseif tag == tag_lv2l    # LV winding phase 2 left
        return Isec2 / Awlv;
    elseif tag == tag_lv2r    # LV winding phase 2 right
        return -Isec2 / Awlv;
    elseif tag == tag_lv3l    # LV winding phase 3 left
        return Isec3 / Awlv * exp(-1im * 2pi/3);
    elseif tag == tag_lv3r    # LV winding phase 3 right
        return -Isec3 / Awlv * exp(-1im * 2pi/3);
    else
        return 0.0 + 0.0im;
    end
end

J0    = fsource ∘ τ; # Source current density
Jeddy = 1im ⋅ omega ⋅ (fsigma ∘ τ) ⋅ mean(uh); # Eddy current density

In [15]:
writevtk(Th, "images/transformer_gridap/transformer6", cellfields=["Az"=>abs(uh), "normB" => Bh, "normJ" => abs(J0 + Jeddy), "mur" => mu_r])

(["images/transformer_gridap/transformer6.vtu"],)

![Flux Density](images/transformer_gridap/transformer6.png)

![Relative Permeability](images/transformer_gridap/transformer6_mu.png)